In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re

class EnterpriseHITLSystem:
    def __init__(self):
        # --- CORE SYSTEM ---
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=10)
        self.memory = {}

        # --- GOVERNANCE & AUDIT COUNTERS ---
        self.stats = {
            "total_decisions": 0,
            "auto_approvals": 0,
            "human_reviews": 0,
            "human_overrides": 0
        }
        self.confidence_threshold = 0.75

    def normalize_text(self, text):
        """Ensures the memory key is consistent regardless of spacing or punctuation."""
        return re.sub(r'[^\w\s]', '', text.lower()).strip()

    def get_ai_analysis(self, text):
        """Generates the AI's technical summary line."""
        try:
            tfidf_matrix = self.vectorizer.fit_transform([text.lower()])
            feature_names = self.vectorizer.get_feature_names_out()
            scores = tfidf_matrix.toarray().flatten()

            confidence = np.mean(np.sort(scores)[::-1][:3]) if len(scores) > 0 else 0.5
            indices = np.argsort(scores)[::-1][:3]
            top_words = [feature_names[i] for i in indices]

            if len(top_words) >= 2:
                ai_line = f"The AI identifies this text as a technical discussion focusing on {top_words[0]} and {top_words[1]}."
            else:
                ai_line = f"The AI detects a focus on {top_words[0] if top_words else 'general topics'}."

            return ai_line, round(float(confidence), 2)
        except:
            return "The AI perceives this as a general descriptive text.", 0.40

    def run_cycle(self):
        print("\n" + "="*65)
        print("ENTERPRISE AI GOVERNANCE LAYER: HITL v2.6 (MEMORY PRIORITY)")
        print("="*65)

        # 1. User Input
        user_text = input("\n[1] Enter your paragraph/prompt:\n> ")
        # Create a normalized key so memory check is perfect
        text_key = self.normalize_text(user_text)
        self.stats["total_decisions"] += 1

        # 2. AI Decision Logic (PRIORITIZING MEMORY)
        if text_key in self.memory:
            # --- MEMORY TRIGGERED ---
            data = self.memory[text_key]
            print(f"\n[2] AI Decision (RECALLED FROM HUMAN MEMORY):")
            print(f"    - Confidence Score      : 1.00 (Verified)")
            print(f"    - Original AI Extraction : {data['ai']}")
            print(f"    - LEARNED Human Decision : {data['human']}") # This gives the human answer

            print("\n[3] STATUS: Pattern recognized. Using previously learned human decision.")
            self.stats["auto_approvals"] += 1
        else:
            # --- NEW INFERENCE ---
            ai_line, confidence_score = self.get_ai_analysis(user_text)
            print(f"\n[2] AI Initial Decision: {ai_line}")
            print(f"    - Confidence Score: {confidence_score}")

            # Confidence-Based Routing
            if confidence_score >= self.confidence_threshold:
                print(f"    - Routing: [HIGH CONFIDENCE] --> AUTO-APPROVING")
                human_note = "Validated AI analysis (Auto-Approved)."
                self.stats["auto_approvals"] += 1
            else:
                print(f"    - Routing: [LOW CONFIDENCE < {self.confidence_threshold}] --> ESCALATING")
                print("\n[3] HUMAN REVIEW REQUIRED: Do you agree or want to override?")
                feedback = input("    Your input (Type 'y' to accept OR enter correction): ")
                self.stats["human_reviews"] += 1

                if feedback.lower() == 'y':
                    human_note = "Accepted by human"
                else:
                    human_note = feedback # Saves your specific sentence
                    self.stats["human_overrides"] += 1
                    print(f"    --> Action: OVERRIDE RECORDED. Memory updated.")

            # Save to Memory for the next time this text appears
            self.memory[text_key] = {"ai": ai_line, "human": human_note}

        # 4. Audit & Adaptation
        override_rate = (self.stats["human_overrides"] / self.stats["total_decisions"]) * 100
        print("\n" + "-"*30)
        print(f"SYSTEM AUDIT LOGS:")
        print(f"Total: {self.stats['total_decisions']} | Overrides: {self.stats['human_overrides']} | Rate: {override_rate:.2f}%")

        if override_rate > 20:
            self.confidence_threshold = 0.85
            print("[ADAPTATION] High Override Rate detected. Escalating threshold to 0.85.")
        print("-"*30)

# --- Start ---
system = EnterpriseHITLSystem()
while True:
    system.run_cycle()
    if input("\nProcess another? (y/n): ").lower() != 'y':
        break


ENTERPRISE AI GOVERNANCE LAYER: HITL v2.6 (MEMORY PRIORITY)

[1] Enter your paragraph/prompt:
>  Carbon is the chemical backbone of all known life on Earth, a remarkably versatile nonmetal with the atomic symbol C and atomic number 6. Its unique ability to form four stable covalent bonds and link with itself in long chains and rings, a property called catenation, allows for the creation of millions of diverse and complex organic compounds, from the carbohydrates in our food to the DNA that carries genetic instructions.

[2] AI Initial Decision: The AI identifies this text as a technical discussion focusing on atomic and catenation.
    - Confidence Score: 0.37
    - Routing: [LOW CONFIDENCE < 0.75] --> ESCALATING

[3] HUMAN REVIEW REQUIRED: Do you agree or want to override?
    Your input (Type 'y' to accept OR enter correction): this paragraph is about the carbon and basically about a catenation process
    --> Action: OVERRIDE RECORDED. Memory updated.

-----------------------------

In [ ]:
""" Carbon is the chemical backbone of all known life on Earth, a remarkably versatile nonmetal with the atomic symbol C and atomic number 6. Its unique ability to form four stable covalent bonds and link with itself in long chains and rings, a property called catenation, allows for the creation of millions of diverse and complex organic compounds, from the carbohydrates in our food to the DNA that carries genetic instructions """